# Importations et variables

In [1]:
from apify_client import ApifyClient
import matplotlib.pyplot as plt
from skimage import io
import ast

import os
from mistralai import Mistral
import json
import functools

In [2]:
model = "mistral-small-latest"
api_key = os.environ["MISTRAL_API_KEY"]
client = Mistral(api_key=api_key)

In [3]:
user_inputs = [
    "I need help choosing an outfit for a weekend brunch with friends. I prefer a casual but put-together look, and I like neutral and earthy tones. My budget is around $100. Can you recommend something stylish yet comfortable?",
    "I have a business meeting next week and want to look professional but not overly formal. My budget is $200. Any suggestions?",
    "I'm going to a beach wedding and need an outfit that’s appropriate for the setting but still dressy. I like pastel colors and flowing fabrics. Budget is $150.",
    "I’m refreshing my wardrobe for the winter season. I’d like some versatile pieces I can layer. My budget is $300, and I like darker tones with occasional pops of color.",
    "I want an outfit for a night out. Something bold and trendy but not too over-the-top. I like jewel tones, and my budget is $120.",
    "I need some ideas for casual work-from-home outfits. They should be comfortable but look good on video calls. My budget is $75 per item.",
    "I’m going to a music festival next month and want an outfit that’s fun, edgy, and practical for a long day outdoors. My budget is $100.",
    "I need a comfortable yet chic outfit for a long-haul flight. I prefer soft fabrics and neutral colors. Budget is $80.",
    "I want to update my gym wardrobe with some trendy athleisure pieces that I can also wear casually. My budget is $150.",
    "I’m attending a formal gala and need an elegant outfit. I prefer classic black or navy, and my budget is $300."
]

# Fonctions utiles

In [4]:
def is_recommandation(user_input):
    prompt = "Analyze the user's input and determine if they are explicitly requesting outfit recommendations. Respond with either True or False, based solely on whether the user’s input suggests they want advice or suggestions for outfits. Do not include any punctuation in your response."
    cur_messages = [
        {"role": "system", "content": prompt},
        {"role":"user", "content":user_input}
    ]
    chat_bool = client.chat.complete(
        model=model,
        messages=cur_messages
    )
    bool_reco = chat_bool.choices[0].message.content
    return bool_reco

"""
def make_prompt(user_input):
    return f
    You are a personal fashion advisor. Your role is to provide tailored fashion advice to users based on their preferences, occasions, and budget. Here's how you should structure your response:

    A list of reference to search on a shopping website. The format must only be a list with [].

    # Example Conversation:
    User Input:
    "I need advice on what to wear for a semi-formal dinner. I like simple, classic styles but with a modern twist. My budget is around $150."

    LLM Response:
    ["slim-fit navy blazers", "non-iron stretch Oxford shirts", "stretch slim-fit chinos", "Leather loafers"]

    # User Input
    {user_input}
"""

def make_prompt(user_input):
    return f"""
    You are a personal fashion advisor. Your role is to provide customized fashion recommendations to users based on their preferences, occasion, and budget. Your response must only include a list of references for items that can be searched on a shopping website, formatted as a list within square brackets [].

    # Example Conversation:
    User Input:
    "I need advice on what to wear for a semi-formal dinner. I like simple, classic styles but with a modern twist. My budget is around $150."

    LLM Response:
    ["slim-fit navy blazers", "non-iron stretch Oxford shirts", "stretch slim-fit chinos", "leather loafers"]

    # User Input:
    {user_input}
    """


def scrap_asos(query, maxItems = 3):
    """ Data scraper from asos.
    Inputs: 
        query (str): search query
        maxItems (int): number of items to scrap
    
    Returns:
        Items (list): list of dictionaries containing the following keys:
            name (str): name of the item
            brandName (str): brand name of the item
            price (str): price of the item
            image (np.array): image of the item
            url (str): url of the item
            gender (str): gender of the item
    """
    
    client = ApifyClient("apify_api_zHwEmmY3hZNab6L57n4NiebpEJDQy42PNRGq")
    
    run_input = {
        "search": query,
        "maxItems": maxItems,
        "endPage": 1,
        "extendOutputFunction": "($) => { return {} }",
        "customMapFunction": "(object) => { return {...object} }",
        "proxy": { "useApifyProxy": True },
    }
    
    run = client.actor("epctex/asos-scraper").call(run_input=run_input)
    
    Items = []
    for item in client.dataset(run["defaultDatasetId"]).iterate_items():
        try :
            # Certains articles n'ont pas de prix d'où la gestion d'erreur
            price = item["variants"][0]["pricing"]["price"]["current"]["text"]
            
            Items.append({
                "name" : item["name"],
                "brandName" : item["brandName"],
                "price" : price,
                "image" : io.imread(item["images"][0]["url"]),
                "gender" : item["gender"],
                "url" : item["url"]
            })
        except:
            pass
        
    return Items

def scrap_asos_outfit(queries, maxItems=3):
    queries = ast.literal_eval(queries)
    outfit = {}
    for i, query in enumerate(queries):
        clothe = scrap_asos(query, maxItems)
        outfit[query] = clothe
    return outfit

def create_first_message(outfit):
    return f"""
    Here are your recommandations :
    {outfit}
    Summarize the different items mentioned by the user in the form of bullet points. 
    Each bullet point must include relevant details about the respective item.

    # Example of response:
    'Introduction to the response'
    - Lightweight denim jacket
        - relevant details
    - Relaxed-fit cotton T-shirt
        - relevant details
    - Slim-fit joggers
        - relevant details
    - White sneakers
        - relevant details
    """

def pipeline_chatbot(prompt, messages=[]):
    # Met le prompt dans le LLM
    messages.append({"role":"user", "content":prompt})
    chat_response = client.chat.complete(
        model=model,
        messages=[messages[-1]]
    )
    # Récupère et process les outfits donnés par le LLM
    queries = chat_response.choices[0].message.content
    print(queries)
    outfit = scrap_asos_outfit(queries, maxItems=1)
    outfit = {query: [{k: v for k, v in item.items() if k != "image"} for item in clothe] for query, clothe in outfit.items()}
    # Créé et envoie un message à envoyer au user
    message_outfit = create_first_message(json.dumps(outfit, indent=4))
    messages.append({"role":"system", "content":message_outfit})
    chat_response = client.chat.complete(
        model = model,
        messages = messages
    )
    print("(ia) >>>", chat_response.choices[0].message.content) # Afficher dans l'interface à la place
    messages.append({"role": "assistant", "content": chat_response.choices[0].message.content})
    
    return messages

def chat_loop(messages=[]):
    while True:
        new_query = input(">>>")
        if new_query == "end":
            break
        print()
        print("(user) >>>", new_query) # Afficher dans l'interface à la place
        bool_reco = is_recommandation(new_query)
        print(bool_reco)
        if bool_reco == "True":
            prompt = make_prompt(new_query)
            messages = pipeline_chatbot(prompt, messages)
        else:
            messages.append({"role": "user", "content": new_query})
            response = client.chat.complete(
                model = model,
                messages = messages
            )
            print()
            print("(ia) >>>",response.choices[0].message.content) # Afficher dans l'interface à la place
            messages.append({"role": "assistant", "content": response.choices[0].message.content})

# Fonction finale

In [5]:
def first_tool_launcher(user_input, messages=[]):
    print(f"(user) >>>{user_input}") # Afficher le message à la place
    bool_reco = is_recommandation(user_input)
    if bool_reco == "True":
        prompt = make_prompt(user_input)
        messages = pipeline_chatbot(prompt, messages)
        chat_loop(messages)
    else:
        messages.append(
            {"role": "user", "content": user_input}
        )
        chat_response = client.chat.complete(
            model=model,
            messages=messages
        )
        response = chat_response.choices[0].message.content
        print(f"(ia) >>> {response}") # Affichage dans l'interface
        messages.append({"role": "assistant", "content": response})
        chat_loop(messages)

In [6]:
first_tool_launcher(user_inputs[0])

(user) >>>I need help choosing an outfit for a weekend brunch with friends. I prefer a casual but put-together look, and I like neutral and earthy tones. My budget is around $100. Can you recommend something stylish yet comfortable?
["denim jacket", "white linen shirt", "beige chino pants", "tan suede Chelsea boots"]
(ia) >>> Here are your recommendations:

- Denim Jacket
  - **Recommended Item:** Wrangler Plus sherpa-lined denim jacket in deep blue
  - **Brand:** Wrangler Plus
  - **Price:** €48.50
  - **Gender:** Women
  - **URL:** [Wrangler Plus Sherpa-Lined Denim Jacket](https://www.asos.com/wrangler-plus/wrangler-plus-sherpa-lined-denim-jacket-in-deep-blue/prd/204892126#colourWayId-204892134)

- Linen Shirt
  - **Recommended Item:** Threadbare co-ord blue and white stripe linen look shirt
  - **Brand:** Threadbare
  - **Price:** €11.09
  - **Gender:** Women
  - **URL:** [Threadbare Linen Look Shirt](https://www.asos.com/threadbare/threadbare-co-ord-blue-and-white-stripe-linen-look

KeyboardInterrupt: 

# Tests

In [6]:
user_input = user_inputs[3]
print(user_input)
first_tool(user_input)

I’m refreshing my wardrobe for the winter season. I’d like some versatile pieces I can layer. My budget is $300, and I like darker tones with occasional pops of color.
(ia) >>> ["Merino wool crew neck sweaters", "Dark denim jeans", "Lightweight down jackets", "Cable-knit scarves", "Wool-blend peacoats", "Chelsea boots", "Thermal henley shirts", "Merino wool beanies"]
(ia) >>> Here are some versatile pieces you can layer for the winter season, focusing on darker tones with occasional pops of color, all within your budget of $300:

### [Merino wool crew neck sweaters, Dark denim jeans, Lightweight down jackets, Cable-knit scarves, Wool-blend peacoats, Thermal henley shirts, Merino wool beanies]

(user) >>> Can you tell me the brands of the different clothes ?

(ia) >>> Certainly! Here’s a summary of the different items and their respective brands:

### Merino Wool Crew Neck Sweaters
- **Brand**: ASOS DESIGN
- **Example Item**: ASOS DESIGN washed knitted muscle rib crew neck jumper in bur

# Autres trucs 

In [ ]:
def queries_scrap_asos(queries):
    outfit = scrap_asos_outfit(queries)
    filtered_outfit = [[{k: v for k, v in item.items() if k != "image"} for item in clothe] for clothe in outfit]
    return json.dumps(filtered_outfit, indent=4)

def pipeline_chat_assos(query_content):
    tools = [
        {
            "type": "function",
            "function": {
                "name": "queries_scrap_asos",
                "description": "Recommendation of an outfit",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "query": {
                            "type": "string",
                            "description": "A list of reference to search on a shopping website. The format must only be a list with [].",
                        }
                    },
                    "required": ["query"],
                },
            },
        },
    ]


    names_to_functions = {
    'queries_scrap_asos': functools.partial(queries_scrap_asos)
    }
    messages = [{"role": "user", "content": query_content}]
    response = client.chat.complete(
        model = model,
        messages = messages,
        tools = tools,
        tool_choice = "any",
    )

    messages.append(response.choices[0].message)

    tool_call = response.choices[0].message.tool_calls[0]
    function_name = tool_call.function.name
    function_params = json.loads(tool_call.function.arguments)
    print("\nfunction_name: ", function_name, "\nfunction_params: ", function_params)

    function_result = names_to_functions[function_name](**function_params)

    messages.append({"role":"tool", "name":function_name, "content":function_result, "tool_call_id":tool_call.id})

    response = client.chat.complete(
        model = model,
        messages = messages
    )
    print("(ia) >>>", response.choices[0].message.content)
    messages.append({"role": "assistant", "content": response.choices[0].message.content})
    return messages

def chat_loop(messages=None):
    if messages == None:
        messages = []

    while True:
        new_query = input(">>>")
        if new_query == "end":
            break
        print()
        print("(user) >>>", new_query)
        messages.append({"role": "user", "content": new_query})
        response = client.chat.complete(
            model = model,
            messages = messages
        )
        print()
        print("(ia) >>>",response.choices[0].message.content)
        messages.append({"role": "assistant", "content": response.choices[0].message.content})

# Beuteu

In [ ]:
outfit = {query: [{k: v for k, v in item.items() if k != "image"} for item in clothe] for query, clothe in outfit.items()}

In [ ]:
def scrap_asos_outfit(queries, maxItems=3):
    queries = ast.literal_eval(queries)
    outfit = []
    for i, query in enumerate(queries):
        clothe = scrap_asos(query, maxItems)
        outfit += clothe
    return outfit

In [13]:
queries = "['black loafers', 'white large shirt']"
outfit = scrap_asos_outfit(queries)

In [14]:
outfit[0].keys()

dict_keys(['name', 'brandName', 'price', 'image', 'gender', 'url'])

In [ ]:
def create_first_message(outfit):
    return f"""
    Here are your recommendations to keep in memory:
    {outfit}
    Respond to the user by confirming that you have relevant recommendations. Do not explicitly reveal the details of the recommendations.
    """

In [ ]:
def process_outfits(outfit):
    for clothe in outfit:
        